In [1]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import Aer
from qiskit import execute
import random
import numpy as np
import sys
sys.path.append("../")
from qmg.utils import MoleculeQuantumStateGenerator, CircuitBuilder
from typing import List, Union
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')



In [2]:
num_heavy_atom = 5
num_sample = 2**15
assert num_heavy_atom >= 2
num_qubits = 4 + (num_heavy_atom-1) * 2
num_clbits = num_heavy_atom * (num_heavy_atom + 1)
num_weights = int(8 + (num_heavy_atom - 2)*(num_heavy_atom + 3) * 3 / 2)
print(num_weights)

data_generator = MoleculeQuantumStateGenerator(num_heavy_atom)
qubits = QuantumRegister(num_qubits)
clbits = ClassicalRegister(num_clbits)
qc = QuantumCircuit(qubits, clbits)
random.seed(1)
weight_vector = [random.random() for _ in range(num_weights)]

def get_classical_register_by_name(qc: QuantumCircuit, name: str):
    for cr in qc.cregs:
        if cr.name == name:
            return cr
    else:
        raise ValueError(f"The {name} is not found in classical registers.")

def softmax_temperature(weight_vector, temperature=0.2):
    weight_vector /= temperature
    exps = np.exp(weight_vector)
    return exps / np.sum(exps)

def controlled_ry(control:int, target:int, digit:float):
    qc.cry(np.pi*digit, control, target)

def reset_qubit(qubit, cbit):
    with qc.if_test((clbits[cbit], 1)):
        qc.x(qubit)

def build_two_atom_circuit(qc, weight_vector, qubits, clbits):
    qc.ry(np.pi * weight_vector[0], 0)
    qc.x(1)
    qc.ry(np.pi * weight_vector[2], 2)
    qc.ry(np.pi * weight_vector[4], 3)
    qc.cx(0, 1)
    controlled_ry(1, 2, weight_vector[3])
    qc.cx(2, 3)
    controlled_ry(0, 1, weight_vector[1])
    qc.cx(1, 2)
    controlled_ry(2, 3, weight_vector[5])

    # measure atom 1 state:
    qc.measure(qubits[0:2], clbits[0:2])
    # measure atom 2 state and save:
    qc.measure(qubits[2:4], clbits[2:4])

    # Add a new atom existence ClassicalRegister:
    atom_existence_CR = ClassicalRegister(bits=clbits[2:4], name="atom_2_existence")
    qc.add_register(atom_existence_CR)
    with qc.if_test((atom_existence_CR, 0)) as else_:
        pass
    with else_:
        qc.ry(np.pi * weight_vector[6], 4)
        qc.x(5)
        qc.cx(4,5)
        controlled_ry(4, 5, weight_vector[7])

    qc.measure(qubits[4:6], clbits[4:6])

def reset_previous_atom_bond_circuit(qc, heavy_idx):
    reset_qubits_index = list(range(2, 2*heavy_idx))
    start_clbit = (heavy_idx - 2)**2 + (heavy_idx - 2)
    reset_clbits_index = list(range(start_clbit, start_clbit+(heavy_idx - 1)*2))
    for qubit_index, clbit_index in zip(reset_qubits_index, reset_clbits_index):
        with qc.if_test((clbits[clbit_index], 1)):
            qc.x(qubit_index)

def build_atom_type_circuit(qc, heavy_idx: int, weight_vector: Union[List[float], np.ndarray]):
    assert len(weight_vector) == 3
    qubit_1_index = 2
    qubit_2_index = 3
    clbit_1_index = (heavy_idx - 1)**2 + (heavy_idx - 1)
    clbit_2_index = clbit_1_index + 1

    register_name = f"atom_{heavy_idx-1}_existence"
    atom_existence_CR = get_classical_register_by_name(qc, register_name)
    with qc.if_test((atom_existence_CR, 0)) as else_:
        pass
    with else_:
        qc.ry(np.pi * weight_vector[0], qubit_1_index)
        qc.ry(np.pi * weight_vector[1], qubit_2_index)
        qc.cx(qubit_1_index, qubit_2_index)
        controlled_ry(4, 5, weight_vector[2])
    qc.measure(qubits[[qubit_1_index,qubit_2_index]], clbits[[clbit_1_index,clbit_2_index]])
    # create new atom existence register
    atom_existence_CR = ClassicalRegister(bits=clbits[[clbit_1_index,clbit_2_index]], name=f"atom_{heavy_idx}_existence")
    qc.add_register(atom_existence_CR)

def build_bond_type_circuit(qc, heavy_idx: int, fixed_weight_vector: Union[List[float], np.ndarray],
                                flexible_weight_vector: Union[List[float], np.ndarray], remove_bond_disconnection=True):
    assert len(fixed_weight_vector) == heavy_idx-1
    assert len(flexible_weight_vector) == 2*(heavy_idx-1)
    qubit_start_index = 4
    qubit_end_index = qubit_start_index + 2*(heavy_idx - 1)
    clbit_start_index = (heavy_idx)**2 - heavy_idx + 2
    clbit_end_index = clbit_start_index + 2*(heavy_idx - 1)

    register_name = f"atom_{heavy_idx}_existence"
    atom_existence_CR = get_classical_register_by_name(qc, register_name)
    with qc.if_test((atom_existence_CR, 0)) as else_:
        pass
    with else_:
        for i in range(heavy_idx-1):
            qc.ry(np.pi * fixed_weight_vector[i], qubit_start_index+2*i+1)
            controlled_ry(qubit_start_index+2*i+1, qubit_start_index+2*i, flexible_weight_vector[2*i]) # < 0.5
            controlled_ry(qubit_start_index+2*i, qubit_start_index+2*i+1, flexible_weight_vector[2*i+1]) # > 0.5
        qc.measure(qubits[qubit_start_index:qubit_end_index], clbits[clbit_start_index:clbit_end_index])
        if remove_bond_disconnection:
            bond_disconnection_CR = ClassicalRegister(bits=clbits[clbit_start_index:clbit_end_index], name=f"bond_{heavy_idx}_connection")
            qc.add_register(bond_disconnection_CR)
            with qc.if_test((bond_disconnection_CR, 0)):
                qc.x(qubit_end_index-1)
                qc.measure(qubits[qubit_end_index-1], clbits[clbit_end_index-1])
    return

build_two_atom_circuit(qc, weight_vector, qubits, clbits)
for heavy_idx in range(3, num_heavy_atom+1):
    atom_type_weights = [random.random() for _ in range(3)]
    bond_type_fixed_weight_vector = np.array([random.random() for _ in range((heavy_idx-1))])
    bond_type_fixed_weight_vector = softmax_temperature(bond_type_fixed_weight_vector, temperature=0.2)
    bond_type_flexible_weight_vector = np.array([random.random()*0.5 for _ in range((heavy_idx-1)*2)])
    bond_type_flexible_weight_vector += np.array([0, 0.5]*(heavy_idx-1))
    print(bond_type_fixed_weight_vector)
    print(bond_type_flexible_weight_vector)

    reset_previous_atom_bond_circuit(qc, heavy_idx)
    build_atom_type_circuit(qc, heavy_idx, atom_type_weights)
    build_bond_type_circuit(qc, heavy_idx, bond_type_fixed_weight_vector,
                                bond_type_flexible_weight_vector, remove_bond_disconnection=True)
simulator = Aer.get_backend('aer_simulator')
results = execute(qc, backend=simulator, shots=num_sample).result()
counts = results.get_counts(qc)
print(counts)
# qc.draw("mpl", style="mpl")
qc.draw("text")

44
[0.11510304 0.88489696]
[0.33515278 0.65168426 0.2937903  0.9412395 ]
[0.02033288 0.05758775 0.92207937]
[0.207157   0.5865037  0.27439938 0.85152038 0.33724292 0.68735151]
[0.40498907 0.21388606 0.34641435 0.03471052]
[0.02174365 0.85169104 0.49159386 0.79659187 0.19679984 0.5851746
 0.25111928 0.99103832]


/tmp/ipykernel_1674551/729006059.py:136: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  results = execute(qc, backend=simulator, shots=num_sample).result()


{'00101000 11 100000 11 0100 11 11 1010': 1, '00100010 10 100000 11 0100 11 10 1010': 1, '00100100 11 100000 10 1000 01 11 1010': 1, '10000000 10 100000 10 1000 11 10 1010': 1, '00100000 01 110000 01 0100 10 11 1010': 1, '00101010 11 110000 11 1000 10 11 1010': 1, '00000010 01 100000 11 1100 01 11 1010': 1, '10000000 01 100000 11 0100 10 11 1011': 1, '00100010 11 100000 10 1000 11 10 1010': 1, '00110000 11 010000 01 0100 01 11 1010': 1, '00100000 11 010000 01 1000 11 10 1010': 1, '00000000 00 100000 11 0100 01 11 1101': 1, '00010010 01 100000 01 1000 11 11 1010': 1, '00100000 11 110000 11 1000 10 11 1010': 1, '10000000 10 010000 01 0110 01 10 1010': 1, '00000100 01 100000 11 1000 01 11 1111': 1, '00100010 10 110000 10 1000 01 10 1010': 1, '00100100 11 100000 01 1000 11 11 1010': 1, '10000000 01 100000 11 0110 01 11 1010': 2, '10000000 11 110000 01 0100 01 10 1010': 1, '00100010 01 110000 01 1000 01 11 1010': 1, '00010010 01 100000 01 0100 01 11 1010': 1, '00000010 01 010000 01 1001 01 

┌─────────────┐                                       ┌─┐              »
 q1_0: ┤ Ry(0.42212) ├──■───────────────────────■────────────┤M├──────────────»
       └────┬───┬────┘┌─┴─┐               ┌─────┴──────┐     └╥┘              »
 q1_1: ─────┤ X ├─────┤ X ├───────■───────┤ Ry(2.6623) ├──■───╫───────────────»
        ┌───┴───┴────┐└───┘┌──────┴──────┐└────────────┘┌─┴─┐ ║               »
 q1_2: ─┤ Ry(2.3995) ├─────┤ Ry(0.80132) ├──────■───────┤ X ├─╫───────■───────»
        ├────────────┤     └─────────────┘    ┌─┴─┐     └───┘ ║ ┌─────┴──────┐»
 q1_3: ─┤ Ry(1.5565) ├────────────────────────┤ X ├───────────╫─┤ Ry(1.4121) ├»
        └────────────┘                        └───┘           ║ └────────────┘»
 q1_4: ───────────────────────────────────────────────────────╫───────────────»
                                                              ║               »
 q1_5: ───────────────────────────────────────────────────────╫───────────────»
                                                              ║               »
 q1_6: ───────────────────────────────────────────────────────╫───────────────»
                                                              ║               »
 q1_7: ───────────────────────────────────────────────────────╫───────────────»
                                                              ║               »
 q1_8: ───────────────────────────────────────────────────────╫───────────────»
                                                              ║               »
 q1_9: ───────────────────────────────────────────────────────╫───────────────»
                                                              ║               »
q1_10: ───────────────────────────────────────────────────────╫───────────────»
                                                              ║               »
q1_11: ───────────────────────────────────────────────────────╫───────────────»
                                                              ║               »
c0: 30/═══════════════════════════════════════════════════════╩═══════════════»
                                                              0               »
«                                                                            »
« q1_0: ─────────────────────────────────────────────────────────────────────»
«       ┌─┐                                                                  »
« q1_1: ┤M├──────────────────────────────────────────────────────────────────»
«       └╥┘┌─┐                                                               »
« q1_2: ─╫─┤M├───────────────────────────────────────────────────────────────»
«        ║ └╥┘┌─┐                                                            »
« q1_3: ─╫──╫─┤M├────────────────────────────────────────────────────────────»
«        ║  ║ └╥┘┌────── ┌──────── ┌───────────┐                    ───────┐ »
« q1_4: ─╫──╫──╫─┤       ┤         ┤ Ry(2.047) ├──■────────■───────        ├─»
«        ║  ║  ║ │ If-0  │ Else-0  └───┬───┬───┘┌─┴─┐┌─────┴──────┐  End-0 │ »
« q1_5: ─╫──╫──╫─┤       ┤         ────┤ X ├────┤ X ├┤ Ry(2.4778) ├        ├─»
«        ║  ║  ║ └──╥─── └────────     └───┘    └───┘└────────────┘ ───────┘ »
« q1_6: ─╫──╫──╫────╫────────────────────────────────────────────────────────»
«        ║  ║  ║    ║                                                        »
« q1_7: ─╫──╫──╫────╫────────────────────────────────────────────────────────»
«        ║  ║  ║    ║                                                        »
« q1_8: ─╫──╫──╫────╫────────────────────────────────────────────────────────»
«        ║  ║  ║    ║                                                        »
« q1_9: ─╫──╫──╫────╫────────────────────────────────────────────────────────»
«        ║  ║  ║    ║                                                        »
«q1_10: ─╫──╫──╫────╫────────────────────────────────────────────────────────»
«        ║  ║  ║    ║                                                        »
«q1_11: ─╫──╫──╫────╫─────────

In [3]:
smiles_dict = {}
num_valid_molecule = 0
for key, value in counts.items():
    key = "".join(key.split())
    smiles = data_generator.QuantumStateToSmiles(data_generator.post_process_quantum_state(key))
    smiles_dict[smiles] = smiles_dict.get(smiles, 0) + value
    if smiles:
        num_valid_molecule += value
validity = num_valid_molecule / num_sample
diversity = (len(smiles_dict.keys()) - 1) / num_sample
print(smiles_dict)
print(validity, diversity)

{None: 7697, 'CCNCC': 1, 'CN1CCN1': 1, 'COCC=N': 1, 'C=C(C)C=N': 1, 'C=CN=NC': 1, 'N#CNNO': 1, 'C#C[N+](=[N-])O': 1, 'C=CN(C)N': 1, 'N=NOOO': 1, 'OC1CC1': 1, 'CNN1C#C1': 1, 'CCC1NO1': 2, 'C=CN(C)O': 1, 'CNNCN': 2, 'N#CCNN': 1, 'C#CONC': 257, 'C=CN(N)O': 4, 'N#CC1CO1': 1, 'NC=CC=O': 1, 'NNCCO': 13, 'NCNN=O': 1, 'NC1CN1O': 1, 'CONC=N': 1, 'C=C=COO': 1, 'C=CC(=O)O': 1, 'NNC(O)O': 1, 'N=CN(O)O': 1, '[N-]=[N+]=NO': 1, 'C=C(O)NN': 1, 'CON1CC1': 1, 'N=COO': 3, 'O=C1CCO1': 3, 'C=C1OO1': 1, 'N#CONN': 2, 'NC1C2CC12': 1, 'c1cn[nH]c#1': 1, 'N=COOO': 1, 'ONONO': 1, 'N=CNNN': 1, 'CC(C)=NO': 1, 'NNOO': 1, 'C=COCO': 1, 'CC=C=CO': 1, 'C=CC=NN': 1, 'OCC1=CC1': 2, 'C#CCN=O': 1, 'CC1NN1N': 2, 'OC1=NC#C1': 2, 'C=COC': 1, 'OCC1CO1': 1, 'C=C(C)NN': 1, 'Nn1c#co1': 7, 'NN=CCO': 2, 'C#CC=CC': 1, 'CC(O)=CO': 2, 'C=C=NON': 1, 'ONOOO': 1, 'CCC(N)N': 3, '[N-]=[N+]1C=CC1': 1, 'CNNNO': 1, 'NCCO': 1, 'CC1NC1=O': 1, 'C=C=NOO': 1, 'CC1C=C1O': 1, 'C[N+](=[N-])ON': 1, 'NCCOO': 41, 'CC=C=NC': 2, 'C=CC#CO': 1, 'NCN=NN': 1, 